## Use Fuzzing Book Resources

In [ ]:
import bookutils
from typing import List, Tuple, Dict, Any
from Fuzzer import RandomFuzzer
from html.parser import HTMLParser
from Coverage import Coverage
import pickle
import hashlib

# Create simple fuzzer
fuzzer = RandomFuzzer(
    min_length=1, max_length=100, char_start=32, char_range=94
)


# Create simple program-under-test
def my_parser(inp: str) -> None:
    parser = HTMLParser()
    parser.feed(inp)


def getTraceHash(cov: Coverage) -> str:
    pickledCov = pickle.dumps(cov.coverage())
    hashedCov = hashlib.md5(pickledCov).hexdigest()
    return hashedCov


# Every program path is a color
inp = "<html>"

with Coverage() as cov:
    my_parser(inp)
getTraceHash(cov)

# **The rarity differs for different coverage**

### Use the fuzzer to create a population

In [ ]:
trials = 50000

population = []
for i in range(trials):
    population.append(fuzzer.fuzz())

population[:3]

### Record the frequency of each coverage

In [ ]:
# TODO: Implement the code to calculate the coverage of the population

############# IMPLEMENT HERE #############
all_coverage_dict = {}
##########################################


### Distribution of the coverage

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

num_unique = len(all_coverage_dict)
print(f"Number of unique traces: {num_unique}")
print(
    f"Top 3 frequencies: {sorted(list(all_coverage_dict.values()), reverse=True)[:3]}"
)
print(f"Bottom 3 frequencies: {sorted(list(all_coverage_dict.values()))[:3]}")
freqs = sorted(list(all_coverage_dict.values()), reverse=True)
# print bar chart of the frequencies
fig, ax = plt.subplots()
sns.barplot(x=list(range(len(freqs))), y=freqs, ax=ax)
# remove the x labels
ax.set_xticks([])
# set log-scale for y-axis
ax.set_yscale("log")
plt.show()

## Most estimators are functions of singletons or doubletons

Singletons: the number of elements that appear only once in the population
Doubletons: the number of elements that appear twice in the population

$$
\Phi_k = \text{the number of elements that appear $k$ times in the population}
$$

Singletons = $\Phi_1$, Doubletons = $\Phi_2$

In [ ]:
# TODO: Compute the singletons and doubletons

############# IMPLEMENT HERE #############
singletons = None
doubletons = None
##########################################

print(
    "The number of coverage elements seen exactly once is "
    + str(len(singletons))
)
print(
    "The number of coverage elements seen exactly twice is "
    + str(len(doubletons))
)

***💡 Go back to the slides***

# **Missing Mass**: What is the probability that the next generated input increases coverage?

### Estimating the propobability of generating a coverage-increasing input

- **Good-Turing** estimator:
$$
\hat{M_0} = \frac{\Phi_1}{n}
$$

In [ ]:
# TODO: Implement the Good-Turing estimator

############# IMPLEMENT HERE #############
estimate = None
##########################################

print(
    "The probability that the next input increases coverage is estimated as "
    + estimate
)

### How do we evaluate this?

In [ ]:
# TODO: Evaluate the estimator empirically

############# IMPLEMENT HERE #############
empirical = None
##########################################

print(
    "The probability that the next input increases coverage is empirically "
    + empirical
)

## Good-Turing allows you to estimate this probability during the campaign.
You only keep track of singletons.

```py
num_inp = 0
all_coverage = set()
singletons = set()
doubletons = set()
for inp in population:
    with Coverage() as cov:
        try:
            my_parser(inp)
        except BaseException:
            pass
    num_inp += 1
    this_coverage = set([getTraceHash(cov)])
    doubletons -= this_coverage                     # Remove this coverage if it was already a doubleton
    doubletons |= (singletons & this_coverage)      # Promote to doubleton if it was a singleton
    singletons -= this_coverage                     # Remove this coverage if it was already a singleton
    singletons |= (this_coverage - all_coverage)    # Add to singleton if it was not seen before
    all_coverage |= this_coverage                   # Add to all coverage seen so far

    estimate = len(singletons) / num_inp
```


## Why does Good-Turing work so well?

Intuitive explanation: "if you observe a new thing at this round, it becomes one of the singletons at the next round."

***💡 Go back to the slides for a little bit deeper explanation :)***

# **Species Richness**: What is the maximum coverage we can achieve?

The estimate of the lower bound of the remaining coverage is
$$
\frac{n - 1}{n}\frac{(\Phi_1)^2}{2\Phi_2}
$$

In [ ]:
# TODO: Estimate the Species Richness

############# IMPLEMENT HERE #############
estimate = None
##########################################

print(
    f"The lower bound of the number of coverages that are still unexplored is {estimate}"
)

### How do we evaluate this?

Let's try to run the fuzzer much longer and see if we can reach the estimate.

In [ ]:
# TODO: Evaluate the estimator by empirically observing 
#       the number of new coverages for a large number of inputs

############# IMPLEMENT HERE #############
extra_coverage = None
##########################################

print(f"Number of new coverages: {len(extra_coverage)}")

***💡 Go back to the slides***

# **Extrapolation**: How much can I discover more when I spend $X$ more time here?

- $\Delta(m)$: the number of new discoveries when $m$ more samples are retrieved.

$$
\hat \Delta(m) = \hat \Phi_0 \left[1 - \left(1 - \frac{\Phi_1}{n\hat \Phi_0 + \Phi_1} \right)^m\right]
$$

### Show the coverage increase with the number of samples.

- Compute the cumulative coverage of the population with size 200,000 + (additional) 200,000
- Extrapolate the number of unique coverage from the point where we have 200,000 samples.

In [ ]:
# First 200,000 inputs
trials = 200000
population = []
for i in range(trials):
    population.append(fuzzer.fuzz())

cumulative_coverage = []
all_coverage = set()
singletons = set()  # Store the number of singletons 
doubletons = set()  # and doubletons for the extrapolation

for inp in population:
    with Coverage() as cov:
        try:
            my_parser(inp)
        except BaseException:
            pass
    this_coverage = set([getTraceHash(cov)])
    doubletons -= this_coverage
    doubletons |= singletons & this_coverage
    singletons -= this_coverage
    singletons |= this_coverage - all_coverage
    all_coverage |= this_coverage
    cumulative_coverage.append(len(all_coverage))

num_singletons = len(singletons)
num_doubletons = len(doubletons)

# Additional 200,000 inputs
extra_trials = 200000
validation = []
for i in range(extra_trials):
    validation.append(fuzzer.fuzz())

for inp in validation:
    with Coverage() as cov:
        try:
            my_parser(inp)
        except BaseException:
            pass
    this_coverage = set([getTraceHash(cov)])
    all_coverage |= this_coverage
    cumulative_coverage.append(len(all_coverage))

fig, ax = plt.subplots()
coverage_ateach_1000trial = [0] + cumulative_coverage[
    999::1000
]  # for every 1000 trials
sub_coverage_until_half = coverage_ateach_1000trial[:201]
sns.lineplot(
    x=range(0, 200001, 1000),
    y=sub_coverage_until_half,
    ax=ax,
)
ax.set_xlim(0, 400000)
ax.set_ylim(0, cumulative_coverage[-1])
ax.set_xticks([0, 100000, 200000, 300000, 400000])
ax.set_xlabel("Number of inputs")
ax.set_ylabel("Number of unique coverages")
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(
    x=range(0, 400001, 1000),
    y=coverage_ateach_1000trial,
    ax=ax,
)
ax.set_xlabel("Number of inputs")
ax.set_ylabel("Number of unique coverages")
plt.show()

In [ ]:
# TODO 3: 

## formula:
## - \Delta(m) = \Phi_0 [1 - (1 - \Phi_1/(n \Phi_0 + \Phi_1) )^m ]
## - \Phi_0 = (n - 1) / n * (\Phi_1^2 / (2 \Phi_2))

############# IMPLEMENT HERE #############
extrapolated = None
##########################################

fig, ax = plt.subplots()
sns.lineplot(
    x=range(0, 200001, 1000),
    y=sub_coverage_until_half,
    ax=ax,
    color="black",
)
sns.lineplot(
    x=range(201000, 400001, 1000),
    y=extrapolated,
    ax=ax,
    color="blue",
    dashes=True,
)

plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1.
sns.lineplot(
    x=range(0, 200001, 1000),
    y=sub_coverage_until_half,
    ax=axes[0],
)
axes[0].set_xlim(0, 400000)
axes[0].set_ylim(0, cumulative_coverage[-1])
axes[0].set_xticks([0, 100000, 200000, 300000, 400000])
axes[0].set_xlabel("Number of inputs")
axes[0].set_ylabel("Number of unique coverages")

# 2.
sns.lineplot(
    x=range(0, 400001, 1000),
    y=coverage_ateach_1000trial,
    ax=axes[1],
)
axes[1].set_xlabel("Number of inputs")
axes[1].set_ylabel("Number of unique coverages")

# 3.
sns.lineplot(
    x=range(0, 200001, 1000),
    y=sub_coverage_until_half,
    ax=axes[2],
    color="black",
)
sns.lineplot(
    x=range(201000, 400001, 1000),
    y=extrapolated,
    ax=axes[2],
    color="blue",
    dashes=True,
)
sns.lineplot(
    x=range(201000, 400001, 1000),
    y=coverage_ateach_1000trial[201:],
    ax=axes[2],
    color="red",
)

plt.show()

***💡 Go back to the slides***

# **Reaching Probability**: What is the probability of reaching a specific program element?

### Let's consider an example of the control flow graph of a program.

Starting from small one with five statements.

In [ ]:
from bn import SimpleBN
import numpy as np

np.random.seed(0)
sbn = SimpleBN(5)
display(sbn.draw())
min_prob = min(sbn.reach_prob_dict.values())
min_nidx = min(sbn.reach_prob_dict, key=sbn.reach_prob_dict.get)
print(f"Min prob: {min_prob} at node {min_nidx}")


Of course, there's some issue with using the bayesian network for CFGs.
- Using the bayesian network for CFGs means the branch conditions are independent.
- Which is not true in general.
  
But, it's a fair approximation for the exercises :)

Edge list:

In [ ]:
display(sbn.edgeidx2edge)

Generate execution

In [ ]:
sbn.gen_obss(1)[0]

In [ ]:
edge_cov = sbn.gen_obss(20)
display(edge_cov)

node_cov = sbn.edgecov2nodecov(edge_cov)
display(node_cov)

covered_nodes = np.logical_or.reduce(node_cov, 0).astype(np.int8)
covered_node_idxs = np.nonzero(covered_nodes)[0]
display(covered_node_idxs)

Let's consider a little bit bigger program.

In [ ]:
sbn = SimpleBN(30)
display(sbn.draw())
min_prob = min(sbn.reach_prob_dict.values())
min_nidx = min(sbn.reach_prob_dict, key=sbn.reach_prob_dict.get)
print(f"Min prob: {min_prob} at node {min_nidx}")


In [ ]:
num_exec = 1000
edge_cov = sbn.gen_obss(num_exec)
node_cov = sbn.edgecov2nodecov(edge_cov)
coverage_freq = np.sum(node_cov, 0)
coverage_freq_dict = {i: coverage_freq[i] for i in range(len(coverage_freq))}
print("Coverage frequencies:")
display(coverage_freq_dict)
covered_nodes = coverage_freq > 0
covered_node_idxs = set(np.nonzero(covered_nodes)[0])
uncovered_node_idxs = set(np.nonzero(1 - covered_nodes)[0])
print("Uncovered nodes:", sorted(uncovered_node_idxs))
for node_idx in sorted(uncovered_node_idxs):
    print(f"Pr({node_idx}) = {sbn.reach_prob_dict[node_idx]}")

### Blackbox estimation

In [ ]:
# Laplace estimator
alpha = 2
esti_laplace = 2 / (num_exec + 4)

# Good-Turing estimator
edge_cov_str = ["".join(map(str, row)) for row in edge_cov]
edge_cov_freq = {edge: edge_cov_str.count(edge) for edge in set(edge_cov_str)}
singletons = {k: v for k, v in edge_cov_freq.items() if v == 1}
est_goodturing = len(singletons) / num_exec

print(f"Estimated Laplace: {esti_laplace}")
print(f"Estimated Good-Turing: {est_goodturing}")

### Structure-aware estimation

For each uncovered statement,
1. find the set of closest reached statements
2. for each of them
   1. find the set of paths from the closest reached statement to the uncovered statement only through the uncovered statements
   2. compute the probability through the paths
3. sum the probabilities

In [ ]:
uncovered_node = sorted(uncovered_node_idxs)[0]
print(f"Node {uncovered_node} is uncovered.")

ancestor_nodes = sbn.get_ancestors(uncovered_node)
print(f"Ancestor nodes: {ancestor_nodes}")

############# IMPLEMENT HERE #############
reached_ancestor_nodes = None
print(f"Reached ancestor nodes: {reached_ancestor_nodes}")
##########################################

In [ ]:
def find_closest_reached_ancestor(
    target_node: int, reached_ancestor_nodes: set, sbn: SimpleBN
) -> int:
    if target_node in reached_ancestor_nodes:
        return {target_node}
    
    ############# IMPLEMENT HERE #############
    closest = set()
    ##########################################
    
    return closest


closest_reached_ancestors = find_closest_reached_ancestor(
    uncovered_node, reached_ancestor_nodes, sbn
)

print(f"Closest reached ancestor nodes: {closest_reached_ancestors}")

In [ ]:
def find_paths(
    source_node: int, target_node: int, avoid_nodes: List[int], sbn: SimpleBN
):
    ############# IMPLEMENT HERE #############
    paths = []
    queue = [[source_node]]
    ##########################################
    
    return paths

for cra in sorted(closest_reached_ancestors):
    paths = find_paths(cra, uncovered_node, covered_node_idxs, sbn)
    print(f"Paths from {cra} to {uncovered_node}: {paths}")

In [ ]:
def compute_prob(path: list, sbn: SimpleBN, n: int, start_freq: int) -> float:
    ############# IMPLEMENT HERE #############
    prob = start_freq / n  # empirical reaching probability of the start node
    ##########################################

    return prob


total_prob = 0
for cra in sorted(closest_reached_ancestors):
    paths = find_paths(cra, uncovered_node, covered_node_idxs, sbn)
    for path in paths:
        prob = compute_prob(path, sbn, num_exec, coverage_freq_dict[cra])
        print(f"Probability of path {path}: {prob}")
        total_prob += prob
print(f"Total probability of reaching node {uncovered_node}: {total_prob}")

In [ ]:
# Another one

uncovered_node = None
print(f"Node {uncovered_node} is uncovered.")

ancestor_nodes = sbn.get_ancestors(uncovered_node)
print(f"Ancestor nodes: {ancestor_nodes}")

reached_ancestor_nodes = ancestor_nodes & covered_node_idxs
print(f"Reached ancestor nodes: {reached_ancestor_nodes}")

closest_reached_ancestors = find_closest_reached_ancestor(
    uncovered_node, reached_ancestor_nodes, sbn
)
print(f"Closest reached ancestor nodes: {closest_reached_ancestors}")

total_prob = 0
for cra in sorted(closest_reached_ancestors):
    paths = find_paths(cra, uncovered_node, covered_node_idxs, sbn)
    print(f"Paths from {cra} to {uncovered_node}: {paths}")
    for path in paths:
        prob = compute_prob(path, sbn, num_exec, coverage_freq_dict[cra])
        print(f"Probability of path {path}: {prob}")
        total_prob += prob
print(f"Total probability of reaching node {uncovered_node}: {total_prob}")

In [ ]:
def structure_aware_esti(
    uncovered_node: int, sbn: SimpleBN, coverage_freq_dict
):
    ancestor_nodes = sbn.get_ancestors(uncovered_node)
    covered_node_idxs = {k for k, v in coverage_freq_dict.items() if v > 0}
    reached_ancestor_nodes = ancestor_nodes & covered_node_idxs
    closest_reached_ancestors = find_closest_reached_ancestor(
        uncovered_node, reached_ancestor_nodes, sbn
    )
    num_exec = sum([v for k, v in coverage_freq_dict.items()])
    total_prob = 0
    for cra in sorted(closest_reached_ancestors):
        remaining_cras = closest_reached_ancestors - {cra}
        paths = find_paths(cra, uncovered_node, remaining_cras, sbn)
        for path in paths:
            prob = compute_prob(path, sbn, num_exec, coverage_freq[cra])
            total_prob += prob
    return total_prob

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "node": sorted(uncovered_node_idxs),
        "true": [sbn.reach_prob_dict[n] for n in sorted(uncovered_node_idxs)],
        "lap": [esti_laplace] * len(uncovered_node_idxs),
        "good": [est_goodturing] * len(uncovered_node_idxs),
        "struct": [
            structure_aware_esti(n, sbn, coverage_freq_dict)
            for n in sorted(uncovered_node_idxs)
        ],
    }
)
display(df)

# draw barplot
df_melt = df.melt(id_vars="node", var_name="method", value_name="prob")
fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x="node", y="prob", hue="method", data=df_melt, ax=ax)
plt.show()

fig, ax = plt.subplots(figsize=(10, 5))
sns.barplot(x="node", y="prob", hue="method", data=df_melt, ax=ax)
ax.set_yscale("log")
plt.show()

***💡 Go back to the slides***